In [ ]:
import csv
import glob
import itertools
import multiprocessing as mp
import os.path
import pickle
import pprint
import time
from io import TextIOWrapper
from multiprocessing import Manager, Pool, Process, Queue
from os import path
from zipfile import ZipFile

import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
from db import NABat_DB
from PIL import Image
from sklearn import metrics
from sklearn.metrics import classification_report

pp = pprint.PrettyPrinter(indent=4)

# Open the database connection.
db = NABat_DB()

# List the bat species we are considering in this model.
species = db.query(
    'select id, species_code, available from species where available = 1;')
class_names = [c[1] for c in species]
print(class_names)

# Given a model and dataset, query our database to return pulse-level predictions.


def get_predictions_pulse(model, draw):
    pulses = db.fastQuery(
        'select count(*) from file f join pulse p on p.file_id = f.id join species s on s.id = f.manual_id where draw = ? and available = 1 and f.grts_id != 0;', (draw,))
    return db.fastQuery("""
    
    with max_conf as (
            select
                p.pulse_id,
                max(p.confidence) as confidence
            from
                prediction p
            where 
                p.model_name = ?
            group by
                p.pulse_id
        )
        
        select
            f.id,
            f.name,
            s.species_code,
            max_conf.confidence
        from 
            max_conf
        join
            prediction p on p.pulse_id = max_conf.pulse_id and p.confidence = max_conf.confidence and p.model_name = ?
        join 
            species s on s.id = p.species_id and available = 1
        join 
            pulse on pulse.id = p.pulse_id
        join
            file f on f.id = pulse.file_id and f.grts_id != 0
        join
            species_grts sl on sl.grts_id = f.grts_id and sl.species_id = p.species_id
        where 
            draw = ?
    """, (model, model, draw)), pulses[0][0]


# Given a model and dataset, query our database to return pulse-level predictions
# aggregated up to the file level, using only pulse predictions with a confidence > min_confidence.
def get_predictions_file(model, draw, min_confidence=0):
    files = db.fastQuery(
        'select count(*) from file f join species s on s.id = f.manual_id where draw = ? and available = 1 and f.grts_id != 0;', (draw,))

    return db.fastQuery("""
    
        with samples as (
            select
                file_id,
                count(*) pc
            from 
                pulse p
            group by 1
        ) 
        , conf as (
            select
                p.file_id,
                pp.species_id,
                count(p.id) num_samples,
                sum(confidence)/(pc) sc
            from
                pulse p
            join
                prediction pp on pp.pulse_id = p.id and pp.model_name = ?
            join 
                samples on samples.file_id = p.file_id
            join 
                species s on s.id = pp.species_id and available = 1
             where 
                pp.confidence > ?
            group by
                p.file_id, pp.species_id
        )
        
        , xx as (
        select
            conf.file_id,
            name,
            s.species_code,
            max(conf.sc) conf,
            f.grts_id,
            s.id as species_id
        from 
            conf
        join
            file f on f.id = conf.file_id and f.grts_id != 0
        join 
            species s on s.id = conf.species_id and available = 1
        
        where 
            draw = ?
        group by 
            1,2 
            
        )
        
    select * from xx 
        join
          species_grts sl on sl.grts_id = xx.grts_id and sl.species_id = xx.species_id
    """, (model, min_confidence, draw)), files[0][0]


# Plot a confusion matrix.
def plot_confusion_matrix(cm, class_names, i, draw):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
       cm (array, shape = [n, n]): a confusion matrix of integer classes
       class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(24, 24))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Identification Rate: {} M-{}".format(draw, i))
    # plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)
                   [:, np.newaxis], decimals=3)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 1.5

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > 0.70 else "black"
        if cm[i, j] > 0.001:
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)
        else:
            plt.text(j, i, '0', horizontalalignment="center", color=color)

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.ioff()
    plt.show()


In [ ]:

# Define the model, dataset, and confidence threshold.
draw = 'test'
model = '1'
thresh = 0.00

# Store aggregated output.
test_labels = []
test_predictions = []
incorrect_confidences = []
correct_confidences = []

predictions, pulse_count = get_predictions_pulse(model, draw)

rejected = pulse_count - len(predictions)

# Loop through each pulse prediction.
for p in predictions:
    if p[-1] >= thresh:
        test_labels.append(p[1].split('/')[-2])
        test_predictions.append(p[2])

        if test_labels[-1] == test_predictions[-1]:
            correct_confidences.append(p[-1])
        else:
            incorrect_confidences.append(p[-1])
    else:
        rejected += 1

print('Rejected {}/{} - {:0.2f}'.format(rejected,
      pulse_count, 100 * (rejected/pulse_count)))


# Print the chart.
cr = classification_report(
    test_labels, test_predictions,  labels=class_names, output_dict=False)
print(cr, '\n\n')

# Plot the matrix.
cr = classification_report(
    test_labels, test_predictions, labels=class_names, output_dict=True)
cr_sorted_keys = sorted(class_names, key=lambda x: (
    cr[x]['recall']), reverse=True)

cm = metrics.confusion_matrix(
    test_labels, test_predictions, labels=cr_sorted_keys, normalize='true')

plot_confusion_matrix(cm, class_names=cr_sorted_keys,
                      i=model, draw=draw.upper())


# Plot the curve, based on confidence and split of correct versus wrong guesses.
plt.close('all')
figure = plt.figure(figsize=(20, 10))

sum_wrong = []
sum_correct = []
x = 0
moving_rate = 0.01
while x < 1:
    x += moving_rate
    sum_wrong.append(len([v for v in incorrect_confidences if v < x]))
    sum_correct.append(len([v for v in correct_confidences if v < x]))

for i in range(len(sum_wrong)):
    if sum_correct[i] > sum_wrong[i]:
        break

auc = i/100
w1 = np.median(incorrect_confidences)
c1 = np.median(correct_confidences)


bins = np.linspace(0, 1, 40)

plt.hist(correct_confidences, bins, alpha=0.5, label='Correct Prediction')
plt.hist(incorrect_confidences, bins, alpha=0.5, label='Wrong Prediction')

plt.vlines(w1, 0, 500, colors='orange', label='Median {:.3f}'.format(w1))
plt.vlines(c1, 0, 500, colors='blue', label='Median {:.3f}'.format(c1))
plt.vlines(auc, 0, 500, linestyles='dashed', colors='red',
           label='Optimal AUC  {:.3f}'.format(auc))

plt.ylabel('Count')
plt.xlabel('Confidence Rate')
plt.title('Validation Set Confidence Rate')

plt.legend(loc='upper left')
plt.show()


In [ ]:
# Define the model, dataset, and confidence threshold.
draw = 'test'
model = '1'
thresh = 0.00

# Store aggregated output.
test_labels = []
test_predictions = []
incorrect_confidences = []
correct_confidences = []

predictions, file_count = get_predictions_file(model, draw, auc)
rejected = file_count - len(predictions)

# Loop through each pulse prediction.
for p in predictions:
    if p[3] >= thresh:
        test_labels.append(p[1].split('/')[-2])
        test_predictions.append(p[2])

        if test_labels[-1] == test_predictions[-1]:
            correct_confidences.append(p[3])
        else:
            incorrect_confidences.append(p[3])
    else:
        rejected += 1

print('Rejected {}/{} - {:0.2f}'.format(rejected,
      file_count, 100 * (rejected/file_count)))

# Print the matrix.
cr = classification_report(
    test_labels, test_predictions,  labels=class_names, output_dict=False)
print(cr, '\n\n')

# Plot the matrix.
cr = classification_report(
    test_labels, test_predictions, labels=class_names, output_dict=True)

cr_sorted_keys = sorted(class_names, key=lambda x: (
    cr[x]['recall']), reverse=True)

cm = metrics.confusion_matrix(
    test_labels, test_predictions, labels=cr_sorted_keys, normalize='true')
plot_confusion_matrix(cm, class_names=cr_sorted_keys,
                      i=model, draw=draw.upper())


# Plot the curve, based on confidence and split of correct versus wrong guesses.
plt.close('all')
figure = plt.figure(figsize=(20, 10))

sum_wrong = []
sum_correct = []
x = 0
moving_rate = 0.01
while x < 1:
    x += moving_rate
    sum_wrong.append(len([v for v in incorrect_confidences if v < x]))
    sum_correct.append(len([v for v in correct_confidences if v < x]))

for i in range(len(sum_wrong)):
    if sum_correct[i] > sum_wrong[i]:
        break

auc = i/100
w1 = np.median(incorrect_confidences)
c1 = np.median(correct_confidences)


bins = np.linspace(0, 1, 40)

plt.hist(correct_confidences, bins, alpha=0.5, label='Correct Prediction')
plt.hist(incorrect_confidences, bins, alpha=0.5, label='Wrong Prediction')

plt.vlines(w1, 0, 500, colors='orange', label='Median {:.3f}'.format(w1))
plt.vlines(c1, 0, 500, colors='blue', label='Median {:.3f}'.format(c1))
plt.vlines(auc, 0, 500, linestyles='dashed', colors='red',
           label='Optimal AUC  {:.3f}'.format(auc))

plt.ylabel('Count')
plt.xlabel('Confidence Rate')
plt.title('Validation Set Confidence Rate')

plt.legend(loc='upper left')
plt.show()
